In [1]:
import os
import requests
import json
import re
import pyttsx3
import random
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [13]:
# ENGLISH TO HINDI TRANSLATOR USING A PRIVATE AZURE SERVICE
subscription_key = '1352efe917df4167b7e990696200e04a'
endpoint =  'https://api.cognitive.microsofttranslator.com/'

path = '/translate?api-version=3.0'
params = '&from=en&to=hi'
constructed_url = endpoint + path + params

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-type': 'application/json',
    'Ocp-Apim-Subscription-Region': 'centralindia',
}

def translate(text):
    body = []
    count = 0
    if isinstance(text, list):
        for sent in text:
            body.append({'text': sent})
            count += len(sent)
            if count>10000:
                body.pop(-1)
                break
    else:
        body.append({'text': text})
    
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    
    if isinstance(text, list):
        try:
            for i in range(len(text)):
                print(text[i], ":", response[i]['translations'][0]['text'])
            return [result['translations'][0]['text'] for result in response]
        except:
            print(json.dumps(response,indent=2))
            print(body)
    else:
        print(text , ":" , response[0]['translations'][0]['text'])
        return response[0]['translations'][0]['text']

translate('test')

test : परीक्षा


'परीक्षा'

In [11]:
# READING ENGLISH DATASET
import pandas as pd
df = pd.read_json('slim intent 2.json')
df.head()

,intent,text,responses,extension,context,entityType,entities
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ...","[Hi human, Hello human, Hola human]","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'GreetingUserRequest', 'clea...",NA,[]
1,GreetingResponse,"[My user is <HUMAN>, This is <HUMAN>, I am <HU...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ...","{'function': 'extensions.gHumans.updateHuman',...","{'in': 'GreetingUserRequest', 'out': '', 'clea...",NA,"[{'entity': 'HUMAN', 'rangeFrom': 3, 'rangeTo'..."
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ...","[Hello, I am great, how are you?, Hello, I am ...","{'function': '', 'entities': False, 'responses...","{'in': '', 'out': 'CourtesyGreetingUserRequest...",NA,[]
3,CourtesyGreetingResponse,"[Good thanks! My user is <HUMAN>, Good thanks!...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ...","{'function': 'extensions.gHumans.updateHuman',...","{'in': 'GreetingUserRequest', 'out': '', 'clea...",NA,"[{'entity': 'HUMAN', 'rangeFrom': 5, 'rangeTo'..."
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who d...","[You are <HUMAN>! How can I help?, Your name i...",{'function': 'extensions.gHumans.getCurrentHum...,"{'in': '', 'out': 'CurrentHumanQuery', 'clear'...",NA,[]


In [7]:
# INTENTS NEED TO BE SPLIT INTO SEPARATE WORDS TO BE PROPERLY TRANSLATED
def camel_case_split(str):
    return ' '.join(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str))

In [9]:
# SPLIT INTENTS AND TRANSLATE TO HINDI
df['intent'] = df['intent'].apply(camel_case_split)
df['intent'] = df['intent'].apply(translate)

Greeting : शुभकामना
Greeting Response : ग्रीटिंग रिस्पांस
Courtesy Greeting : सौजन्य अभिवादन
Courtesy Greeting Response : सौजन्य ग्रीटिंग रिस्पांस
Current Human Query : वर्तमान मानव क्वेरी
Name Query : नाम क्वेरी
Real Name Query : असली नाम क्वेरी
Time Query : टाइम क्वेरी
Thanks : धन्यवाद
Not Talking U : बात नहीं यू
Understand Query : क्वेरी को समझें
Shutup : शुटअप
Good Bye : अलविदा
Courtesy Good Bye : सौजन्य गुड बाय
Who Am I : मैं कौन हूँ
Clever : चतुर
Self Aware : आत्म जागरूक


In [10]:
df['responses'] = df['responses'].apply(translate)
df['text'] = df['text'].apply(translate)

Hi human : हाय मानव
Hello human : नमस्ते मानव
Great! Hi <HUMAN>! How can I help? : शानदार! हाय <HUMAN>! मैं कैसे मदद कर सकता हूं?
Good! Hi <HUMAN>, how can I help you? : अच्छा! हाय <HUMAN>, मैं आपकी मदद कैसे कर सकता हूं?
Cool! Hello <HUMAN>, what can I do for you? : ठंडक! नमस्ते <HUMAN>, मैं तुम्हारे लिए क्या कर सकता हूं?
OK! hi <HUMAN>, what can I do for you? : ठीक है! हाय <HUMAN>, मैं तुम्हारे लिए क्या कर सकता हूं?
Hello, I am great, how are you? : नमस्ते, मैं महान हूं, तुम कैसे हो?
Hello, I am good thank you, how are you? : हैलो, मैं अच्छा हूं धन्यवाद, आप कैसे हैं?
Hi, good thank you, how are you? : हाय, अच्छा धन्यवाद, आप कैसे हैं?
Great! Hi <HUMAN>! How can I help? : शानदार! हाय <HUMAN>! मैं कैसे मदद कर सकता हूं?
Good! Hi <HUMAN>, how can I help you? : अच्छा! हाय <HUMAN>, मैं आपकी मदद कैसे कर सकता हूं?
Cool! Hello <HUMAN>, what can I do for you? : ठंडक! नमस्ते <HUMAN>, मैं तुम्हारे लिए क्या कर सकता हूं?
OK! hi <HUMAN>, what can I do for you? : ठीक है! हाय <HUMAN>, मैं तुम्हारे लिए 

In [ ]:
df.head()

In [25]:
translate("Bye! See you soon.")

Bye! See you soon. : अलविदा! जल्दी ही मिलेंगे।


'अलविदा! जल्दी ही मिलेंगे।'